In [2]:
import numpy as np
import pandas as pd
from gurobipy import * 
import gurobipy as gp

In [3]:
dis_zip = pd.read_csv('https://raw.githubusercontent.com/iamzhenxp/E4418-Transportation-Anlytics-Logistic/FInal-project/FInal%20Project/zipcode.csv', encoding='unicode_escape')
demand = pd.read_csv('https://raw.githubusercontent.com/iamzhenxp/E4418-Transportation-Anlytics-Logistic/FInal-project/FInal%20Project/totaldistance.csv', encoding='unicode_escape')
scooter = pd.read_csv('https://raw.githubusercontent.com/iamzhenxp/E4418-Transportation-Anlytics-Logistic/FInal-project/FInal%20Project/Minneapolis_data_coordinate.csv', encoding='unicode_escape')
zip_cor = pd.read_csv('https://gist.githubusercontent.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data', encoding='unicode_escape')
#demand = pd.read_csv('', encoding='unicode_escape')

In [4]:
dis_zip

,Zipcode,55414.0,55401.0,55405.0,55403.0,55404.0,55402.0,55415.0,55408.0,55454.0,...,55411.0,55455.0,55418.0,55406.0,55410.0,55416.0,55407.0,55409.0,55419.0,55430.0
0,55414.0,1.0,4.5,10.7,6.8,4.9,6.4,3.7,9.6,3.7,...,7.8,2.1,6.8,5.7,19.2,16.5,8.7,9.6,13.2,21.8
1,55401.0,4.1,1.0,4.9,2.2,4.0,2.3,1.9,7.2,5.0,...,3.5,3.9,5.6,7.6,16.2,10.7,7.8,7.2,10.8,14.1
2,55405.0,10.2,5.0,1.0,3.4,6.1,4.6,8.4,6.7,8.9,...,4.1,8.7,13.7,11.0,13.1,5.4,10.8,9.8,13.4,16.0
3,55403.0,7.8,2.2,3.4,1.0,2.7,1.3,2.7,4.5,6.5,...,4.1,6.3,11.3,8.6,15.7,9.3,7.1,6.1,9.6,15.7
4,55404.0,5.7,3.6,7.0,2.7,1.0,2.3,1.8,5.2,2.3,...,7.3,4.3,9.2,5.0,14.9,7.4,3.9,5.3,8.8,18.2
5,55402.0,6.5,1.4,4.8,1.6,2.7,1.0,1.4,6.2,4.3,...,3.8,3.9,6.3,6.6,15.8,10.6,7.2,6.2,9.8,14.6
6,55415.0,3.6,1.9,7.9,4.0,2.1,3.6,1.0,6.8,1.8,...,8.1,2.2,7.1,5.7,16.4,13.7,5.9,6.8,10.3,19.0
7,55408.0,9.7,7.7,7.3,4.4,5.0,6.4,7.9,1.0,8.4,...,7.5,8.2,13.2,6.2,5.3,4.7,3.9,2.0,4.3,18.4
8,55454.0,3.4,5.0,9.0,5.2,2.5,4.7,2.3,7.9,1.0,...,9.2,2.2,7.4,5.1,17.5,14.8,5.0,7.9,11.4,20.2
9,55413.0,3.6,3.2,11.2,7.3,5.4,6.9,4.6,10.1,4.9,...,4.6,5.3,3.4,8.9,19.7,17.0,9.2,10.1,13.6,14.6


In [5]:
me = pd.merge(dis_zip,demand,left_on='Zipcode',right_on='Zipcode')
demand = me[['Zipcode','TripDistance']]
demand

,Zipcode,TripDistance
0,55414.0,70357651
1,55401.0,96611863
2,55405.0,5043493
3,55403.0,67718543
4,55404.0,6914123
5,55402.0,44146832
6,55415.0,30088935
7,55408.0,19088572
8,55454.0,3169588
9,55413.0,6128215


In [6]:
n = len(dis_zip)
n

20

In [7]:
FC = 4350
UTC = 1
k=0.01
CAP = 1000
HC =0.1

In [8]:
A = np.zeros([40,440])
for i in range(n):
    for j in range(n):
        A[i,i*n+j]=k*demand['TripDistance'][i]/365
    A[i,420+i]=-1
for i in range(n):
    for j in range(n):
        A[20+i,20*j+i]=1
pd.DataFrame(A).to_csv('look.csv')
ba = np.zeros(40)
ba[-20:] = np.ones(20)
ba

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.])

In [9]:
B = np.zeros([420,440])
for i in range(n):
    B[i,420+i]=1
for i in range(n):
    for j in range(n):
        B[20+n*i+j,n*i+j] =1
        B[20+n*i+j,400+i]=-1
pd.DataFrame(B).to_csv('look2.csv')
bb = np.zeros(420)
bb[:20] = CAP
bb

array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,

In [10]:
c = np.zeros(440)
c[:400]=np.array(dis_zip.iloc[:,1:]*365*2).reshape(400)
c[400:420] = FC
c[420:440] = HC

In [11]:
m = Model('battery')
# variables
xvar = {}
for i in range(n):
    for j in range(n):
        xvar[20*i+j] = m.addVar(name='x'+str(i)+','+str(j),ub=1)
for j in range(n):
    xvar[n*n+j] = m.addVar(name='y'+str(j),vtype=GRB.BINARY)
for j in range(n):
    xvar[n*n+n+j] = m.addVar(name='z'+str(j))

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-14


In [12]:
len(xvar)

440

In [13]:
for i in range(40):
    m.addConstr(sum(A[i,j]*xvar[j] for j in range(440)) == ba[i], name = 'rowA'+str(i))

In [14]:
for i in range(420):
    m.addConstr(sum(B[i,j]*xvar[j] for j in range(440)) <= bb[i], name = 'rowB'+str(i))

In [15]:
for i in range(20):
    m.addConstr((1-xvar[400+i])*xvar[420+i] == 0)

In [16]:
m.setObjective( sum(c[j]*xvar[j] for j in range(440)) , GRB.MINIMIZE)

In [17]:
m.update()

In [18]:
m.write('optimization.lp')
f = open('optimization.lp','r') 

In [19]:
Lines = f.readlines()
f.close()
for line in Lines: print(line)

\ Model battery

\ LP format - for model browsing. Use MPS format to capture full model detail.

Minimize

  730 x0,0 + 3285 x0,1 + 7811 x0,2 + 4964 x0,3 + 3577 x0,4 + 4672 x0,5

   + 2701 x0,6 + 7008 x0,7 + 2701 x0,8 + 2555 x0,9 + 5694 x0,10

   + 1533 x0,11 + 4964 x0,12 + 4161 x0,13 + 14016 x0,14 + 12045 x0,15

   + 6351 x0,16 + 7008 x0,17 + 9636 x0,18 + 15914 x0,19 + 2993 x1,0

   + 730 x1,1 + 3577 x1,2 + 1606 x1,3 + 2920 x1,4 + 1679 x1,5 + 1387 x1,6

   + 5256 x1,7 + 3650 x1,8 + 2336 x1,9 + 2555 x1,10 + 2847 x1,11

   + 4088 x1,12 + 5548 x1,13 + 11826 x1,14 + 7811 x1,15 + 5694 x1,16

   + 5256 x1,17 + 7884 x1,18 + 10293 x1,19 + 7446 x2,0 + 3650 x2,1

   + 730 x2,2 + 2482 x2,3 + 4453 x2,4 + 3358 x2,5 + 6132 x2,6 + 4891 x2,7

   + 6497 x2,8 + 8103 x2,9 + 2993 x2,10 + 6351 x2,11 + 10001 x2,12

   + 8030 x2,13 + 9563 x2,14 + 3942 x2,15 + 7884 x2,16 + 7154 x2,17

   + 9782 x2,18 + 11680 x2,19 + 5694 x3,0 + 1606 x3,1 + 2482 x3,2

   + 730 x3,3 + 1971 x3,4 + 949 x3,5 + 1971 x3,6 + 3285 x3

In [20]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 460 rows, 440 columns and 1640 nonzeros
Model fingerprint: 0x886bf469
Model has 20 quadratic constraints
Variable types: 420 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 107 rows and 20 columns
Presolve time: 0.01s
Presolved: 353 rows, 420 columns, 1313 nonzeros
Variable types: 400 continuous, 20 integer (20 binary)
Found heuristic solution: objective 104047.66440

Root relaxation: objective 6.405060e+04, 92 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

In [21]:
var = pd.DataFrame(m.x)
x = pd.DataFrame(np.array(var.iloc[:400]).reshape([20,20]))
y = var.iloc[400:420]
z = var.iloc[420:440]
z

,0
420,0.000000
421,0.000000
422,552.711562
423,0.000000
424,1000.000000
425,0.000000
426,0.000000
427,0.000000
428,0.000000
429,671.585205


In [22]:
x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.27905,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
y.index = range(20)
y['Zipcode']=demand.Zipcode
y.columns = ['y_value','Zipcode']
y['serveby']=[9,9,2,2,4,4,4,17,4,9,2,4,9,17,17,2,17,17,17,19]
y

<ipython-input-23-f6c02c5bf822>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Zipcode']=demand.Zipcode
<ipython-input-23-f6c02c5bf822>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['serveby']=[9,9,2,2,4,4,4,17,4,9,2,4,9,17,17,2,17,17,17,19]


,y_value,Zipcode,serveby
0,-0.0,55414.0,9
1,-0.0,55401.0,9
2,1.0,55405.0,2
3,-0.0,55403.0,2
4,1.0,55404.0,4
5,-0.0,55402.0,4
6,-0.0,55415.0,4
7,-0.0,55408.0,17
8,-0.0,55454.0,4
9,1.0,55413.0,9


In [24]:
serve_zip = y[['Zipcode','serveby']]
serve_zip

,Zipcode,serveby
0,55414.0,9
1,55401.0,9
2,55405.0,2
3,55403.0,2
4,55404.0,4
5,55402.0,4
6,55415.0,4
7,55408.0,17
8,55454.0,4
9,55413.0,9


In [25]:
bss_loc = pd.merge(scooter,serve_zip,left_on='Zipcode',right_on='Zipcode',how='left')
bss_loc = bss_loc[['start_lat','start_lon','Zipcode','serveby']]
bss_loc

,start_lat,start_lon,Zipcode,serveby
0,44.981964,-93.234606,55414.0,9
1,44.981536,-93.236780,55414.0,9
2,44.980936,-93.266058,55401.0,9
3,44.956413,-93.296596,55405.0,2
4,44.968794,-93.281348,55403.0,2
...,...,...,...,...
149742,44.982561,-93.269574,55401.0,9
149743,44.985777,-93.269574,55401.0,9
149744,44.969863,-93.280080,55403.0,2
149745,44.988167,-93.277601,55401.0,9


In [26]:
zip_cor

,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135
...,...,...,...
33139,99923,56.002315,-130.041026
33140,99925,55.550204,-132.945933
33141,99926,55.138352,-131.470424
33142,99927,56.239062,-133.457924


In [27]:
bss = y.loc[y.y_value==1]
bss = pd.merge(bss,zip_cor,left_on='Zipcode',right_on='ZIP',how='left')
bss

,y_value,Zipcode,serveby,ZIP,LAT,LNG
0,1.0,55405.0,2,55405,44.972669,-93.301675
1,1.0,55404.0,4,55404,44.961996,-93.261208
2,1.0,55413.0,9,55413,44.999463,-93.241677
3,1.0,55409.0,17,55409,44.930326,-93.281477
4,1.0,55430.0,19,55430,45.067676,-93.302089


In [28]:
from gmplot import gmplot

In [29]:
lats = bss['LAT'].tolist()
lons = bss['LNG'].tolist()
gmap = gmplot.GoogleMapPlotter(lats[0], lons[0], 13)
gmap.apikey="..."
gmap.scatter(lats, lons, '# FF0000', size = 40, marker = True)
lats2 = bss_loc.loc[bss_loc.serveby==2]['start_lat'].tolist()
lons2 = bss_loc.loc[bss_loc.serveby==2]['start_lon'].tolist()
gmap.heatmap(lats2, lons2,radius=5,opacity = 0.7,gradient=[(0, 0, 255, 0), (0, 255, 0, 0.9), (255, 0, 0, 1)])
lats2 = bss_loc.loc[bss_loc.serveby==4]['start_lat'].tolist()
lons2 = bss_loc.loc[bss_loc.serveby==4]['start_lon'].tolist()
gmap.heatmap(lats2, lons2,radius=5,opacity = 0.7,gradient=[(0, 0, 255, 0), (0, 255, 0, 0.9), (0, 255, 0, 1)])
lats2 = bss_loc.loc[bss_loc.serveby==9]['start_lat'].tolist()
lons2 = bss_loc.loc[bss_loc.serveby==9]['start_lon'].tolist()
gmap.heatmap(lats2, lons2,radius=5,opacity = 0.7,gradient=[(0, 0, 255, 0), (0, 255, 0, 0.9), (0, 0, 255, 1)])
lats2 = bss_loc.loc[bss_loc.serveby==17]['start_lat'].tolist()
lons2 = bss_loc.loc[bss_loc.serveby==17]['start_lon'].tolist()
gmap.heatmap(lats2, lons2,radius=5,opacity = 1,gradient=[(0, 0, 255, 0), (0, 255, 0, 0.9), (170, 170, 100, 1)])
lats2 = bss_loc.loc[bss_loc.serveby==19]['start_lat'].tolist()
lons2 = bss_loc.loc[bss_loc.serveby==19]['start_lon'].tolist()
gmap.heatmap(lats2, lons2,radius=100,opacity = 0.5,gradient=[(0, 0, 255, 0), (0, 255, 0, 0.9), (200, 170, 230, 1)])

In [30]:
#gmap.heatmap(lats, lons,radius=8,opacity = 0.5)
gmap.draw("bss_opt.html")

/Users/zhendongyang/opt/anaconda3/lib/python3.8/site-packages/gmplot/color.py:194: UserWarning: Color '# FF0000' isn't supported.
  warnings.warn("Color '%s' isn't supported." % color)


In [36]:
y.loc[y.serveby==19]

,y_value,Zipcode,serveby
19,1.0,55430.0,19
